In [0]:
import nltk
import pandas as pd

In [2]:
df = pd.read_csv('quora_duplicate_questions.tsv', sep='\t')
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0.0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0.0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0.0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0.0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0.0


In [3]:
df.drop(['id', 'qid1', 'qid2'], axis=1, inplace=True)
df.shape
df.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0.0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0.0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0.0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0.0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0.0


In [4]:
# Check for any null values
print(df.isnull().sum())
df = df.dropna()
print("\nAfter preprocessing \n-------------------")
print(df.isnull().sum())

question1       0
question2       0
is_duplicate    1
dtype: int64

After preprocessing 
-------------------
question1       0
question2       0
is_duplicate    0
dtype: int64


In [5]:
def print10Ques():
    a = 0 
    for i in range(a,a+10):
        print(df.question1[i])
        print(df.question2[i])
        print()

print10Ques()

What is the step by step guide to invest in share market in india?
What is the step by step guide to invest in share market?

What is the story of Kohinoor (Koh-i-Noor) Diamond?
What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?

How can I increase the speed of my internet connection while using a VPN?
How can Internet speed be increased by hacking through DNS?

Why am I mentally very lonely? How can I solve it?
Find the remainder when [math]23^{24}[/math] is divided by 24,23?

Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?
Which fish would survive in salt water?

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?

Should I buy tiago?
What keeps childern active and far from phone and video games?

How can I be a good geologist?
What should I do to be a great geologist?

When do you use シ instea

In [7]:
# from pyspark.sql import functions, types
# from pyspark.sql.functions import concat, col, lit, udf
from nltk.stem.snowball import SnowballStemmer
import re
import spacy

# add spacing at the beginning and end of string
def addPadding(string):
    return ' '+string+' '
    
def removeStopWords(text):
    stop_words = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This']
    
    text = text.split()
    text = [w for w in text if not w in stop_words]
    text = " ".join(text)
    return text

def runStemmer(text):
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

def runLemmatizer(text):
    sp = spacy.load('en')
    text = sp(text)
    text = " ".join([token.lemma_ for token in text])
    return text
    
# normalize text
def normalizeText(text, remStopWords=True, stemWords=False):
    text = str(text)
    
    # Handle known symbols like $, %, &
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)
    
    # remove remaining punctuation from the text
    text = re.sub(r'[^\w\s]','', text)
    
    # replace nonascii characters by word 'nonascii'
    text = re.sub('[^\x00-\x7F]+', addPadding('nonascii'), text)
    
    if remStopWords:
        text = removeStopWords(text)
    
    if stemWords:
        text = runStemmer(text)
        
    text = runLemmatizer(text)
    text = re.sub(r'[^\w\s]','', text)
    
    # finally, convert all words in text to lower case
    result = text.lower()

    return result

df['ques1'] = df['question1'].apply(normalizeText, args=(True, False))
df['ques2'] = df['question2'].apply(normalizeText, args=(True, False))

a = 0 
for i in range(a,a+10):
    print(df.question1[i])
    print(df.ques1[i])
    print(df.question2[i])
    print(df.ques2[i])
    print()

What is the step by step guide to invest in share market in india?
step by step guide invest in share market in india
What is the step by step guide to invest in share market?
step by step guide invest in share market

What is the story of Kohinoor (Koh-i-Noor) Diamond?
story kohinoor kohinoor diamond
What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?
would happen indian government steal kohinoor kohinoor diamond back

How can I increase the speed of my internet connection while using a VPN?
how can pron increase speed pron internet connection use vpn
How can Internet speed be increased by hacking through DNS?
how can internet speed be increase by hack dns

Why am I mentally very lonely? How can I solve it?
why be pron mentally very lonely how can pron solve pron
Find the remainder when [math]23^{24}[/math] is divided by 24,23?
find remainder when math2324math divide by 2423

Which one dissolve in water quikly sugar, salt, methane and carbon di oxi

In [8]:
df.head()

,question1,question2,is_duplicate,ques1,ques2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0.0,step by step guide invest in share market in i...,step by step guide invest in share market
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0.0,story kohinoor kohinoor diamond,would happen indian government steal kohinoor ...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0.0,how can pron increase speed pron internet conn...,how can internet speed be increase by hack dns
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0.0,why be pron mentally very lonely how can pron ...,find remainder when math2324math divide by 2423
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0.0,one dissolve in water quikly sugar salt methan...,fish would survive in salt water


In [10]:
# we just need ques1, ques2 and is_duplicate
quora_df = pd.DataFrame({'question1':df['ques1'], 'question2':df['ques2'], 'is_duplicate': df['is_duplicate']})
quora_df.head()

,is_duplicate,question1,question2
0,0.0,step by step guide invest in share market in i...,step by step guide invest in share market
1,0.0,story kohinoor kohinoor diamond,would happen indian government steal kohinoor ...
2,0.0,how can pron increase speed pron internet conn...,how can internet speed be increase by hack dns
3,0.0,why be pron mentally very lonely how can pron ...,find remainder when math2324math divide by 2423
4,0.0,one dissolve in water quikly sugar salt methan...,fish would survive in salt water


In [0]:
# save csv file for preprocessed quora dataset
quora_df.to_csv('preprocessed_Quora.csv', sep='\t')